In [1]:
import pandas as pd
import numpy as np
from imblearn.over_sampling import SMOTE
from imblearn.under_sampling import RandomUnderSampler
from imblearn.combine import SMOTEENN
from sklearn.model_selection import train_test_split, KFold
from sklearn.ensemble import RandomForestClassifier
from sklearn.tree import DecisionTreeClassifier
import xgboost as xgb
from sklearn.metrics import accuracy_score

https://xgboost.readthedocs.io/en/stable/tutorials/model.html

In [2]:
df = pd.read_pickle(r'C:\Users\usuario\Documents\12° SEMESTRE\TCC2\data\data_pre_processed.pkl')
del df['Class_BMI'], df['ID']
df.head()

,HeartDisease,BMI,Smoking,AlcoholDrinking,Stroke,PhysicalHealth,MentalHealth,DiffWalking,Sex,AgeCategory,Race,Diabetic,PhysicalActivity,GenHealth,SleepTime,Asthma,KidneyDisease,SkinCancer
0,0,16.60,1,0,0,3.0,30.0,0,1,7,0,1,1,3,5.0,1,0,1
1,0,20.34,0,0,1,0.0,0.0,0,1,12,0,0,1,3,7.0,0,0,0
2,0,26.58,1,0,0,20.0,30.0,0,0,9,0,1,1,1,8.0,1,0,0
3,0,24.21,0,0,0,0.0,0.0,0,1,11,0,0,0,2,6.0,0,0,1
4,0,23.71,0,0,0,28.0,0.0,1,1,4,0,0,1,3,8.0,0,0,0


In [3]:
X = df.drop('HeartDisease', axis=1)
y = df[['HeartDisease']]

In [4]:
X_treino, X_teste, y_treino, y_test = train_test_split(X,y, random_state=42)

X_treino.shape, X_teste.shape, y_treino.shape, y_test.shape

((239846, 17), (79949, 17), (239846, 1), (79949, 1))

In [5]:
classificador_RF = RandomForestClassifier()
classificador_DT = DecisionTreeClassifier()
classificador_XGB = xgb.XGBClassifier()

In [6]:
def validacao_cruzada(modelo, X, y, balanceamento):
    kfold = KFold(n_splits=5)

    acuracias = []

    for idx, (idx_treino, idx_validacao) in enumerate(kfold.split(X)):
        X_split_treino = X.iloc[idx_treino,:]
        y_split_treino = y.iloc[idx_treino,:]

        if balanceamento=='Oversampling':
            bal = SMOTE(random_state=42)
            X_split_treino, y_split_treino =  bal.fit_resample(X_split_treino, y_split_treino)
        elif balanceamento=='Undersampling':
            bal = RandomUnderSampler(random_state=42)
            X_split_treino, y_split_treino = bal.fit_resample(X_split_treino, y_split_treino)
        elif balanceamento =='SMOTEENN':
            bal = SMOTEENN(random_state=42, sampling_strategy=0.5, n_jobs=-1)
            X_split_treino, y_split_treino = bal.fit_resample(X_split_treino, y_split_treino)
        else:
            print('Os tipos de balanceamento são: Oversampling, Undersampling e SMOTEENN ')
            break
        modelo.fit(X_split_treino, y_split_treino.values.flatten())

        X_split_validacao = X.iloc[idx_validacao,:]
        y_split_validacao = y.iloc[idx_validacao,:]

        predicao_validacao = modelo.predict(X_split_validacao)
        acuracia_split = accuracy_score(y_split_validacao,predicao_validacao)

        acuracias.append(acuracia_split)
        print(f'Acurácia do split {idx}: {acuracia_split}')
    return modelo

Avaliando o modelo Decision Tree

In [22]:
modelo = validacao_cruzada(classificador_DT, X_treino, y_treino, 'Oversampling')

Acurácia do split 0: 0.8245361684386074
Acurácia do split 1: 0.8258041651900185
Acurácia do split 2: 0.8257833183931289
Acurácia do split 3: 0.8226562988596802
Acurácia do split 4: 0.8275136025349705


In [23]:
pred = modelo.predict(X_teste)
ac = accuracy_score(y_test,pred)
print(f'Acurácia do teste: {ac}')

Acurácia do teste: 0.8229621383632065


In [13]:
validacao_cruzada(classificador_DT, X_treino, y_treino, 'Undersampling')

Acurácia do split 0: 0.6734834271419637
Acurácia do split 1: 0.678917634305489
Acurácia do split 2: 0.6763743250849507
Acurácia do split 3: 0.674102024223978
Acurácia do split 4: 0.6766870270382955


In [14]:
validacao_cruzada(classificador_DT, X_treino, y_treino, 'SMOTE')

Os tipos de balanceamento são: Oversampling, Undersampling e SMOTEENN 


In [15]:
validacao_cruzada(classificador_DT, X_treino, y_treino, 'SMOTEENN')

Acurácia do split 0: 0.7739420471127788
Acurácia do split 1: 0.7775021367966812
Acurácia do split 2: 0.7744793512476807
Acurácia do split 3: 0.7739581813254394
Acurácia do split 4: 0.7753966103108257


Avaliando o modelo Random Forest

In [14]:
validacao_cruzada(classificador_RF, X_treino, y_treino, 'Oversampling')

Acurácia do split 0: 0.8558474046278924
Acurácia do split 1: 0.857949925993871
Acurácia do split 2: 0.856428109820926
Acurácia do split 3: 0.8564072630240364
Acurácia do split 4: 0.8579916195876504


In [7]:
validacao_cruzada(classificador_RF, X_treino, y_treino, 'Undersampling')

Acurácia do split 0: 0.7223681467583907
Acurácia do split 1: 0.7242177239467156
Acurácia do split 2: 0.7210698576163772
Acurácia do split 3: 0.720840542850591
Acurácia do split 4: 0.7213825595697221


RandomForestClassifier()

In [8]:
validacao_cruzada(classificador_RF, X_treino, y_treino, 'SMOTEENN')

Acurácia do split 0: 0.7920366895976652
Acurácia do split 1: 0.792324209385228
Acurácia do split 2: 0.790072755321145
Acurácia do split 3: 0.7884467051637516
Acurácia do split 4: 0.7916779586816486


RandomForestClassifier()

Avaliação o modelo XGBOOST

In [17]:
validacao_cruzada(classificador_XGB, X_treino, y_treino, 'Oversampling')

Acurácia do split 0: 0.8339378778403168
Acurácia do split 1: 0.8304321540995226
Acurácia do split 2: 0.8296816694114949
Acurácia do split 3: 0.8299318309741708
Acurácia do split 4: 0.8324334466009298


In [18]:
validacao_cruzada(classificador_XGB, X_treino, y_treino, 'Undersampling')

Acurácia do split 0: 0.7343339587242026
Acurácia do split 1: 0.7336404761408409
Acurácia do split 2: 0.7330984594217098
Acurácia do split 3: 0.7327440638745857
Acurácia do split 4: 0.7327649106714753


In [19]:
validacao_cruzada(classificador_XGB, X_treino, y_treino, 'SMOTEENN')

Acurácia do split 0: 0.7830310610798416
Acurácia do split 1: 0.7800037524234401
Acurácia do split 2: 0.7776272175780191
Acurácia do split 3: 0.7788363317976192
Acurácia do split 4: 0.7793158081260815
